In [1]:
import os
os.chdir(r"C:/Users/LukeStrange/Code/constituencies")
from pipelines.util import *

if __name__ == '__main__':
    charging_device_stats = read_data("raw-data/electric-vehicle-charging-statistics-july-2023.ods",
                                      sheet_name='1a',
                                      engine='odf',
                                      skiprows=2)
    charging_device_stats = charging_device_stats[charging_device_stats['Local Authority / Region Code'].str.match('^(E06|E07|E08|E09|S12|W06|N09)([0-9]{6})')]
    charging_device_stats = charging_device_stats[charging_device_stats.columns[charging_device_stats.columns.str.contains('Total Charging Devices|Local Authority / Region Code')]]

    charging_grant_stats_workplace = read_data('raw-data/electric-vehicle-charging-device-grant-scheme-statistics.ods',
                                               sheet_name='3',
                                               engine='odf',
                                               skiprows=2)
    
    charging_grant_stats_home = read_data('raw-data/electric-vehicle-charging-device-grant-scheme-statistics.ods',
                                               sheet_name='10',
                                               engine='odf',
                                               skiprows=2)
    
    charging_grant_stats_workplace = charging_grant_stats_workplace[charging_grant_stats_workplace['ONS LA Code'].str.match('^(E06|E07|E08|E09|S12|W06|N09)([0-9]{6})')]
    charging_grant_stats_home = charging_grant_stats_home[charging_grant_stats_home['ONS LA Code'].str.match('^(E06|E07|E08|E09|S12|W06|N09)([0-9]{6})')]
    
    charging_device_stats = charging_device_stats.loc[:, ['Local Authority / Region Code', 'Apr-23 (Total Charging Devices)']].rename(columns={'Local Authority / Region Code': 'LA22CD', 'Apr-23 (Total Charging Devices)': 'public-2023-04'})
    charging_grant_stats_workplace = charging_grant_stats_workplace.loc[:, ['ONS LA Code', 'Sockets Installed']].rename(columns={'ONS LA Code': 'LA22CD', 'Sockets Installed': 'workplace-2023-04'})
    charging_grant_stats_home = charging_grant_stats_home.loc[:,['ONS LA Code', '2022']].rename(columns={'ONS LA Code': 'LA22CD', '2022': 'home-2022'})

    merged_df = charging_grant_stats_workplace.merge(charging_grant_stats_home.merge(charging_device_stats, on='LA22CD'), on='LA22CD')
    merged_df.replace('x', 'NaN', inplace=True)
    merged_df['public-2023-04'] = merged_df['public-2023-04'].astype(float)
    merged_df['total'] = merged_df[['workplace-2023-04', 'home-2022', 'public-2023-04']].sum(axis=1)



In [5]:
merged_df

,LA22CD,workplace-2023-04,home-2022,public-2023-04,total
0,E06000047,378,535,240.0,1153.0
1,E06000005,103,105,73.0,281.0
2,E06000001,43,70,27.0,140.0
3,E06000002,86,98,58.0,242.0
4,E06000057,231,464,264.0,959.0
...,...,...,...,...,...
369,N09000006,27,48,35.0,110.0
370,N09000007,106,185,20.0,311.0
371,N09000008,57,107,27.0,191.0
372,N09000009,76,57,26.0,159.0


In [7]:
import json
import numpy as np
# load the lookup table
with open('lookups/lookup-LAD22CD-PCON22CD.json') as f:
    lookup = json.load(f)

# create an empty dataframe to put the data into
df = pd.DataFrame(columns=['PCON22CD', 'workplace-2023-04', 'home-2022', 'public-2023-04', 'total'])
df.set_index('PCON22CD', inplace=True)


# iterate through each column of the LA_Code dataframe
for la, c in lookup.items():
    # get a key and value pair for each constituency and its overlap
    for constituency_code, overlap in c.items():
        # if constituency doesnt exist, loop over columns and set all to 0.
        if constituency_code not in df.index.to_list():
            for cname in df.columns.to_list():
                df.loc[constituency_code, cname] = 0.0
        for col in df.columns.to_list():
            try:
                # calculate the share of charging devices in each constituency
                share = merged_df.loc[merged_df.LA22CD==la, col].values[0]*overlap
                df.loc[constituency_code, col] += share
            except:
                print(share)
                
df = df.apply(np.floor)
#df.to_csv(os.path.join(TRANSPORT_DATA_DIR, 'ev_charging_points.csv'))
df

,workplace-2023-04,home-2022,public-2023-04,total
PCON22CD,,,,
E14000733,43,70,27,140
E14000819,46,52,31,130
E14000820,68,121,67,257
E14000891,12,32,17,62
E14000970,140,148,73,362
...,...,...,...,...
W07000054,47,142,67,256
W07000055,38,89,61,189
W07000056,37,87,61,185


In [4]:
list1 = [1,2,3]
list2 = [4,5,6]
for i, j in zip(list1, list2):
    print('i', i, 'j', 

SyntaxError: incomplete input (4283979423.py, line 4)

In [ ]:
df = pd.DataFrame(data=np.zeros((1,5)), columns=['PCON22CD', 'workplace-2023-04', 'home-2022', 'public-2023-04', 'total'])
df.set_index('PCON22CD', inplace=True)
df

,workplace-2023-04,home-2022,public-2023-04,total
PCON22CD,,,,
0.0,0.0,0.0,0.0,0.0
